# CNN 3 - Getting started with Convolutional layers
- Dataset:
    - https://www.kaggle.com/shaunthesheep/microsoft-catsvsdogs-dataset
- The dataset isn't deep-learning-compatible by default, here's how to preprocess it:
- Code: https://github.com/fenago/deeplearning/blob/main/tensorflow/008_CNN_001_Working_With_Image_Data.ipynb
    
**Before you start**
- I got TensorFlow errors during training because a couple of images were corrupted
- Before continuing, please delete the following images:
    - `data\train\cat\666.jpg`
    - `data\train\dog\11702.jpg`

In [ ]:
import os
import pathlib
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(42)

from PIL import Image, ImageOps
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

<br>

## Normalizing image data
- Let's load in a sample image:

In [ ]:
img1 = Image.open('data/train/cat/1.jpg')
display(img1)

- And check it's shape

In [ ]:
np.array(img1).shape

- It's 281 pixels wide, 300 pixels tall, and has 3 color channels
- Let's load in another image and see if the same applies:

In [ ]:
img2 = Image.open('data/train/dog/0.jpg')
display(img2)

In [ ]:
np.array(img2).shape

- The second image is much larger
- Neural network doesn't like that - it expects images (arrays) of identical sizes
- You'll see later how to resize them on the fly
- First, let's see how a single image looks like when represented as an array:

In [ ]:
np.array(img2)

- It's in a range between 0 and 255 for every single color channel (red, green, and blue)
- Neural networks prefer a range between 0 and 1
- We can translate it to that range by dividing each element of an array by 255.0:

In [ ]:
np.array(img2) / 255.0

- That's the only argument we'll pass to the TensorFlow's ImageDataGenerator - rescaling
- There are others available, and we'll cover them in a couple of notebooks when learning data augmentation

<br>

## Data loaders
- You can use the `ImageDataGenerator` class from TensorFlow to specify how the image data will be generated
- We'll only apply rescaling - 1 / 255.0
- We'll do this for both trianing and validation images:

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0)

- You can now use this generator to load in data from a directory
- Specify the directory path, and a siye to which each image will be resized
    - 224x224 works well with neural networks, especially with transfer learning models (more on these in a couple of notebooks)
- Set `class_mode='categorical'`, since we have two distinct classes
- Set `batch_siye=64` or anything you want, it represents the number of images _shown_ to a neural network at once
- The `seed` parameter is here so you can get the same images as I did:

In [ ]:
train_data = train_datagen.flow_from_directory(
    directory='data/train/',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=64,
    seed=42
)

- There are 20030 images in the training folder divided into two classes - as reported by the loader
- The `train_data` is basically a Python generator object
- You can call `next()` on it to get the first batch:

In [ ]:
first_batch = train_data.next()

- Each batch contains images and labels
- Let's check the shape:

In [ ]:
first_batch[0].shape, first_batch[1].shape

- So, a single batch contains 64 images, each being 224 pixels wide and tall with 3 color channels
- There are 64 corresponding labels, each is an array of two elements - probability of an image being a cat (0) ond a dog (1)

<br>

## Visualizing a single batch
- It's always recommended to visalize your data
- The `visualize_batch()` function, well, visualizes a single batch
- There are 64 images in the batch, so the function plots a grid of 8x8 images:

In [ ]:
def visualize_batch(batch: tf.keras.preprocessing.image.DirectoryIterator):
    n = 64
    num_row, num_col = 8, 8
    fig, axes = plt.subplots(num_row, num_col, figsize=(3 * num_col, 3 * num_row))
    
    for i in range(n):
        img = np.array(batch[0][i] * 255, dtype='uint8')
        ax = axes[i // num_col, i % num_col]
        ax.imshow(img)
        
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_batch(batch=first_batch)

- Some of them look a bit weird due to change in the aspect ratio, but we should be fine
- Let's reset the data loaders, as we called `next()` before:

In [ ]:
train_data = train_datagen.flow_from_directory(
    directory='data/train/',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=64,
    seed=42
)

valid_data = valid_datagen.flow_from_directory(
    directory='data/validation/',
    target_size=(224, 224),
    class_mode='categorical',
    batch_size=64,
    seed=42
)

<br>

## Training a Convolutional model
- Just like with regular ANN's (Dense layers), Convolutional Neural Networks boil down to experimentation
- You can't know beforehand how many Convolutional layers you'll need, what's the ideal number of filters for each, and what's the optimal kernel size
- Convolutional layers are usually followed by a Pooling layer, to reduce the image size
- When finished with Convolutional layers, make sure to add a Flatten layer
- Add Dense layers as you normally would from there
- Keep in mind the ouput layer and the loss function
    - Use softmax activation at output, as sigmoid only works when you have a single output node
    - Track loss through categorical cross entropy
- We'll train the model for 10 epochs, which is completely random:

In [ ]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), input_shape=(224, 224, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model_1.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

history_1 = model_1.fit(
    train_data,
    validation_data=valid_data,
    epochs=10
)

- 71.23% accuracy after 10 epochs
- Does doubling the number of filters in our single Convolutional layers make a difference?

In [ ]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape=(224, 224, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model_2.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

history_2 = model_2.fit(
    train_data,
    validation_data=valid_data,
    epochs=10
)

- Maybe, but the model generally doesn't look like it's learning
- Let's add another Convolutional layer
    - Keep in mind: Only the first convolutional layer needs the `input_shape` parameter

In [ ]:
model_3 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), input_shape=(224, 224, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model_3.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

history_3 = model_3.fit(
    train_data,
    validation_data=valid_data,
    epochs=10
)

- Much better - we're at 75% now on the validation set
- Let's use this model to make predictions

<br>

## Making predictions on new images
- You have to apply the same preprocessing operations to the unseen images
- I've forgot to do so many times on my job, and it results in some wierd and uncertain predictions (small difference between prediction probabilities)
- We'll declare a `prepare_single_image()` function which resizes an image to 224x224 and rescales it to a 0-1 range:

In [ ]:
def prepare_single_image(img_path: str) -> np.array:
    img = Image.open(img_path)
    img = img.resize(size=(224, 224))
    return np.array(img) / 255.0

- Let's test it on a single image:

In [ ]:
single_image = prepare_single_image(img_path='data/test/cat/10018.jpg')
single_image

- And now let's make a single prediction
- Note the `reshape()` function - try removing it and see what happens
- There's an easier way, and you'll see it in a bit

In [ ]:
single_prediction = model_3.predict(single_image.reshape(-1, 224, 224, 3))
single_prediction

- These are basically prediction probabilities
- The model almost 100% certain that the class at index 0 is present on the image
- Remember: 0 = cat, 1 = dog
- You can use the argmax function to get the index where the value of an array is the highest:

In [ ]:
single_prediction.argmax()

- Let+s make predictions for an entire folder of images
- First for the cats
- The top two variables will track how many predictions were made, and how many of these were correct
- Note the `expand_dims()` function - it's an alternative to `reshape()`
    - You can use either
- Prediction fails on some images probably because they are corrupted, so wrap the code inside a `try .. except` block:

In [ ]:
num_total_cat, num_correct_cat = 0, 0

for img_path in pathlib.Path.cwd().joinpath('data/test/cat').iterdir():
    try:
        img = prepare_single_image(img_path=str(img_path))
        pred = model_3.predict(tf.expand_dims(img, axis=0))
        pred = pred.argmax()
        num_total_cat += 1
        if pred == 0:
            num_correct_cat += 1
    except Exception as e:
        continue

- Total predictions made:

In [ ]:
num_total_cat

- Accuracy for cats:

In [ ]:
num_correct_cat / num_total_cat

- Not too bad - let's do the same for dogs:

In [ ]:
num_total_dog, num_correct_dog = 0, 0

for img_path in pathlib.Path.cwd().joinpath('data/test/dog').iterdir():
    try:
        img = prepare_single_image(img_path=str(img_path))
        pred = model_3.predict(tf.expand_dims(img, axis=0))
        pred = pred.argmax()
        num_total_dog += 1
        if pred == 1:
            num_correct_dog += 1
    except Exception as e:
        continue

In [ ]:
num_total_dog

In [ ]:
num_correct_dog / num_total_dog

- Overall, we have a much more accurate model than when we were only using Dense layers
- This is just a tip of the iceberg
    - We haven't explored data augmentation and transfer learning
    - You wouldn't believe how much these will increase the accuracy